<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/aapl_trading_dataset_pipeline_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# ✅ Import Libraries
import pandas as pd
import numpy as np
import yfinance as yf
from google.colab import drive

# ✅ Mount Google Drive
drive.mount('/content/drive')

# ✅ Download Stock Data (Apple - AAPL)
df = yf.download("AAPL", period="720d", interval="1h")
df.reset_index(inplace=True)


def clean_stock_data(df):
    """
    Cleans and standardizes stock data columns to:
    ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
    """

    # ✅ 1. Fix Multi-Index Columns
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    # ✅ 2. Standardize Time Column
    time_columns = ['Datetime', 'datetime', 'Date', 'date', 'timestamp']
    for col in time_columns:
        if col in df.columns:
            df.rename(columns={col: 'Datetime'}, inplace=True)
            break
    if 'Datetime' not in df.columns:
        df.reset_index(names='Datetime', inplace=True)

    # ✅ 3. Standardize Price Columns
    column_map = {
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'adjclose': 'Close',  # Adjusted Close fallback
        'volume': 'Volume'
    }

    df.columns = [column_map.get(col.lower(), col) for col in df.columns]

    # ✅ 4. Select Only Relevant Columns
    required_columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
    df = df[[col for col in required_columns if col in df.columns]]

    # ✅ 5. Convert Datetime Format
    df['Datetime'] = pd.to_datetime(df['Datetime'], errors='coerce')

    # ✅ 6. Drop Missing or Corrupted Rows
    df.dropna(subset=['Datetime', 'Close'], inplace=True)

    # ✅ 7. Sort by Time
    df.sort_values(by='Datetime', inplace=True)

    # ✅ 8. Reset Index
    df.reset_index(drop=True, inplace=True)

    print(f"✅ Cleaned Data Columns: {df.columns}")
    print(df.head())

    return df
# ✅ Step 2: Clean the Data
df = clean_stock_data(df)



# ==========================
# 📌 Calculate Indicators (from your code)
# ==========================
# ✅ Step 2: Clean the Data Using Provided Function
df = clean_stock_data(df)

# ✅ Step 3: Fix Potential Multi-Index Issues (Optional Safety Check)
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

# ✅ Calculate ATR (Average True Range)
df['High-Low'] = df['High'] - df['Low']
df['High-Close'] = abs(df['High'] - df['Close'].shift(1))
df['Low-Close'] = abs(df['Low'] - df['Close'].shift(1))
df['True_Range'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
df['ATR'] = df['True_Range'].rolling(window=14).mean().iloc[:, 0] if isinstance(df['True_Range'].rolling(window=14).mean(), pd.DataFrame) else df['True_Range'].rolling(window=14).mean()
# Ensure ATR is a Series, not a DataFrame
df['ATR'] = df['ATR'].iloc[:, 0] if isinstance(df['ATR'], pd.DataFrame) else df['ATR']


df.drop(columns=['High-Low', 'High-Close', 'Low-Close', 'True_Range'], inplace=True)

# ✅ Dynamic Stop-Loss and Take-Profit
df['Stop_Loss'] = df['Close'] - (1.2 * df['ATR'])
df['Take_Profit'] = df['Close'] + (2.0 * df['ATR'])

# ✅ Exponential Moving Averages (EMA)
df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()
df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()

# ✅ Simple Moving Averages (SMA)
df['SMA_20'] = df['Close'].rolling(window=20).mean()
df['SMA_50'] = df['Close'].rolling(window=50).mean()

# ✅ MACD Calculation (Momentum Indicator)
df['MACD_Line'] = df['Close'].ewm(span=12, adjust=False).mean() - df['Close'].ewm(span=26, adjust=False).mean()
df['MACD_Signal'] = df['MACD_Line'].ewm(span=9, adjust=False).mean()
df['MACD_Hist'] = df['MACD_Line'] - df['MACD_Signal']

# ✅ RSI Calculation (Overbought/Oversold)
delta = df['Close'].diff()
gain = delta.where(delta > 0, 0).rolling(window=14).mean()
loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))

# ✅ ADX Calculation (Trend Strength)
df['+DM'] = np.where((df['High'].diff() > df['Low'].diff()) & (df['High'].diff() > 0), df['High'].diff(), 0)
df['-DM'] = np.where((df['Low'].diff() > df['High'].diff()) & (df['Low'].diff() > 0), df['Low'].diff(), 0)
df['+DI'] = 100 * df['+DM'].rolling(window=14).mean() / df['ATR']
df['-DI'] = 100 * df['-DM'].rolling(window=14).mean() / df['ATR']
df['ADX'] = abs(df['+DI'] - df['-DI']).rolling(window=14).mean()
df.drop(columns=['+DM', '-DM'], inplace=True)

# ✅ Volume Average
df['Volume_Avg'] = df['Volume'].rolling(window=20).mean()

# ✅ Volume Change Rate (First Define It)
df['Volume_Change'] = df['Volume'].pct_change()

# ✅ Volume Change Moving Average (10-period average)
df['Volume_Change_MA'] = df['Volume_Change'].rolling(window=10).mean()

# ✅ Volume Change Ratio (Rate of change in volume from previous)
df['Volume_Change_Ratio'] = df['Volume_Change'] / df['Volume_Change'].shift(1)

# ✅ Relative Volume (Volume compared to 20-period average)
df['Relative_Volume'] = df['Volume'] / df['Volume_Avg']


# ✅ Trailing Stop Calculation
df['Trailing_Stop'] = np.minimum(
    df['Close'] * 0.985,          # 2% below price
    df['Close'] - (df['ATR'] * 0.3)  # Tighter ATR stop
)

# ==========================
# 📌 Generate Buy & Sell Signals (Labels) Based on Your Rules
# ==========================

# ✅ Buy Signal Logic
df['Buy_Signal'] = np.where(
    (df['RSI'] < 60) &
    (df['EMA_10'] > df['EMA_50']) &
    ((df['MACD_Line'] > df['MACD_Signal']) | (df['MACD_Line'].diff() > 0)) &
    (df['Volume'] > (0.4 * df['Volume_Avg'])) &
    (df['ADX'] > 18),
    1, 0
)

# ✅ Sell Signal Logic
df['Sell_Signal'] = np.where(
    ((df['EMA_10'] < df['EMA_50']) & (df['RSI'] > 60)) |
    ((df['MACD_Line'] < df['MACD_Signal']) & (df['RSI'] > 65)) |
    (df['Close'] < df['Trailing_Stop']) |
    ((df['Volume'] > 0.5 * df['Volume_Avg']) & (df['ADX'] > 20)),
    1, 0
)

# ✅ Debug Sell Signal Logic (from your code)
df['Sell_Signal_Debug'] = np.where(
    ((df['MACD_Hist'] < 0.5) | (df['MACD_Line'] < df['MACD_Signal'])) &
    (df['RSI'] < 55) &
    (df['ADX'] > 15) &
    ((df['Close'] < df['Trailing_Stop']) | (df['EMA_10'] < df['EMA_50'])) &
    (df['Volume'] > 0.5 * df['Volume_Avg']),
    1, 0
)

# ==========================
# 📌 Save Final Labeled Dataset for Supervised Learning
# ==========================

# ✅ Select Important Columns for ML
final_columns = [
    'Datetime', 'Open', 'High', 'Low', 'Close', 'Volume',
    'EMA_10', 'EMA_50', 'SMA_20', 'SMA_50',
    'RSI', 'MACD_Line', 'MACD_Signal', 'MACD_Hist',
    'ATR', 'ADX', 'Volume_Avg', 'Trailing_Stop',
    'Buy_Signal', 'Sell_Signal', 'Sell_Signal_Debug'
]

# ✅ Filter Columns
labeled_df = df[final_columns]

# ✅ Add Future Return Columns
df['Future_Close'] = df['Close'].shift(-10)  # Price 10 periods ahead
df['Return'] = (df['Future_Close'] - df['Close']) / df['Close']  # % Return

# ✅ Add Classification Labels (1 if price increases by 2%, else 0)
df['Target'] = np.where(df['Return'] > 0.02, 1, 0)  # Buy signal if 2% return

# ✅ Multi-Class Target Labeling
df['Target'] = np.select(
    [
        df['Return'] > 0.02,  # Strong Uptrend
        df['Return'] < -0.02  # Strong Downtrend
    ],
    [1, -1],
    default=0  # No significant movement
)




# ✅ Drop Future Columns Before Saving for Model Training
model_columns = [col for col in labeled_df.columns if col not in ['Future_Close', 'Return']]
labeled_df = df[model_columns]

# ✅ Add Time-Based Features
df['Hour'] = pd.to_datetime(df['Datetime']).dt.hour
df['DayOfWeek'] = pd.to_datetime(df['Datetime']).dt.dayofweek

# ✅ Add Session Labels (e.g., Pre-market, Regular, After-hours)
df['Session'] = np.where(
    (df['Hour'] >= 9) & (df['Hour'] <= 16), 'Regular',
    np.where((df['Hour'] < 9), 'Pre-market', 'After-hours')
)

# ✅ Rolling Volatility (standard deviation of returns)
df['Volatility'] = df['Close'].pct_change().rolling(window=20).std()

# ✅ MACD Crossover Signal
df['MACD_Crossover'] = np.where(df['MACD_Line'] > df['MACD_Signal'], 1, 0)

# ✅ Multi-Class Target Labeling
df['Multi_Class_Target'] = np.select(
    [
        df['Return'] > 0.02,  # Strong Uptrend
        df['Return'] < -0.02  # Strong Downtrend
    ],
    [1, -1],
    default=0  # No significant movement
)



# ✅ Final Dataset Columns

final_columns = [
    'Datetime', 'Open', 'High', 'Low', 'Close', 'Volume',
    'EMA_10', 'EMA_50', 'SMA_20', 'SMA_50',
    'RSI', 'MACD_Line', 'MACD_Signal', 'MACD_Hist',
    'ATR', 'Volatility', 'Volume_Avg', 'Volume_Change', 'Relative_Volume',
    'ADX', 'Trailing_Stop','Buy_Signal', 'Sell_Signal', 'Sell_Signal_Debug',
    'Return', 'Target','Multi_Class_Target','Hour', 'DayOfWeek', 'Session',
    'MACD_Crossover',
    'Future_Close'
]


# ✅ Remove any infinite values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# ✅ Drop rows with missing values
df.dropna(inplace=True)

print(f"✅ Final dataset shape: {df.shape}")


# ✅ Filter and Save
labeled_df = df[final_columns]
csv_path = '/content/drive/My Drive/aapl_labeled_trading_dataset_v2.csv'
labeled_df.to_csv(csv_path, index=False)

print(f"✅ Enhanced labeled dataset saved: {csv_path}")


# ✅ Save to CSV in Google Drive
csv_path = '/content/drive/My Drive/aapl_labeled_trading_dataset.csv'
labeled_df.to_csv(csv_path, index=False)

print(f"✅ Labeled dataset saved to Google Drive: {csv_path}")
print(f"🚀 Total Buy Signals: {labeled_df['Buy_Signal'].sum()}")
print(f"🚀 Total Sell Signals: {labeled_df['Sell_Signal'].sum()}")
print(f"🚀 Total Debug Sell Signals: {labeled_df['Sell_Signal_Debug'].sum()}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[*********************100%***********************]  1 of 1 completed


✅ Cleaned Data Columns: Index(['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
                   Datetime        Open        High         Low       Close  \
0 2022-04-04 13:30:00+00:00  174.554993  178.210007  174.479996  177.360001   
1 2022-04-04 14:30:00+00:00  177.360001  177.839996  176.729996  177.645004   
2 2022-04-04 15:30:00+00:00  177.649994  177.830597  177.110001  177.660004   
3 2022-04-04 16:30:00+00:00  177.664993  177.684998  176.919998  177.229599   
4 2022-04-04 17:30:00+00:00  177.229996  177.615005  177.020004  177.490005   

     Volume  
0  22098890  
1  10059372  
2   6683479  
3   5749349  
4   5992490  
✅ Cleaned Data Columns: Index(['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
                   Datetime        Open        High         Low       Close  \
0 2022-04-04 13:30:00+00:00  174.554993  178.210007  174.479996  177.360001   
1 2022-04-04 14:30:00+00:00  177.360001  177.839996  176.729996  177.645004   


In [6]:
df.head()

,Datetime,Open,High,Low,Close,Volume,ATR,Stop_Loss,Take_Profit,EMA_10,...,Sell_Signal_Debug,Future_Close,Return,Target,Hour,DayOfWeek,Session,Volatility,MACD_Crossover,Multi_Class_Target
49,2022-04-13 13:30:00+00:00,167.520004,168.899994,166.770004,168.690002,15711444,1.408214,167.000146,171.506430,167.876788,...,0,166.559998,-0.012627,0,13,2,Regular,0.008165,1,0
50,2022-04-13 14:30:00+00:00,168.699997,170.000000,168.285004,169.690002,9751613,1.463572,167.933716,172.617146,168.206463,...,0,166.669296,-0.017801,0,14,2,Regular,0.008220,1,0
51,2022-04-13 15:30:00+00:00,169.699997,171.000000,169.600098,170.440094,10069224,1.506064,168.632817,173.452223,168.612578,...,0,165.865005,-0.026843,-1,15,2,Regular,0.008226,1,-1
52,2022-04-13 16:30:00+00:00,170.449997,171.000000,170.279999,170.559998,6851724,1.492493,168.769006,173.544983,168.966654,...,0,165.279999,-0.030957,-1,16,2,Regular,0.008141,1,-1
53,2022-04-13 17:30:00+00:00,170.570007,170.593597,169.949997,170.279999,6078766,1.484179,168.498984,173.248356,169.205444,...,0,166.149994,-0.024254,-1,17,2,After-hours,0.008082,1,-1
